Need to Install

<ol>
  <li>pip install  fuzzywuzzy</li>
</ol>

In [ ]:
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from tqdm import tqdm
import re

# Load the first CSV file into a Pandas dataframe
df1 = pd.read_csv("./Data/sample_data_Scrape_Result.csv")
# Load the second CSV file into a second Pandas dataframe
df2 = pd.read_csv("./Data/Globe-institution_data.csv")

In [ ]:
df1.head(1)


In [ ]:
# List to hold the results
results = []
# Set to hold the 'Acc' values for which matches have already been found
matched_ids = set()
# Iterate over the columns in df1
for column in ['Center Names', 'Submitted_by', 'Biosample_Submission']:
    # Iterate over the rows in df1
    for index, row in tqdm(df1.iterrows(), total=df1.shape[0], desc='Rows', leave=False):
        # Check if this 'Acc' has already been matched
        if row['Acc'] in matched_ids:
            continue
        # Convert the series to a string
        string = str(row[column])
        # Check if the column contains a comma
        if ',' in string:
            # Split the string into two parts based on the comma
            parts = string.split(',')
            # Find the closest match in df2 for each part
            matches = [process.extractOne(part.strip(), df2['Institution'], scorer=fuzz.token_sort_ratio) for part in parts]
            # Check if there is a match for both parts
            if all(match and match[1] >= 70 for match in matches):
                # Get the indices of the matching rows in df2
                match_indices = [df2[df2['Institution'] == match[0]].index[0] for match in matches]
                # Concatenate the rows from df1 and df2 and add to the results list
                result = pd.concat([df1.loc[[index], :].reset_index(drop=True), df2.loc[match_indices, :].reset_index(drop=True)], axis=1)
                result['Score'] = sum(match[1] for match in matches) / len(matches)
                results.append(result)
                # Add the 'Acc' value to the set of matched ids
                matched_ids.add(row['Acc'])
        else:
            # Find the closest match in df2
            match = process.extractOne(string, df2['Institution'], scorer=fuzz.token_sort_ratio)
            if match and match[1] >= 70:
                # Get the index of the matching row in df2
                match_index = df2[df2['Institution'] == match[0]].index[0]
                # Concatenate the rows from df1 and df2 and add to the results list
                result = pd.concat([df1.loc[[index], :].reset_index(drop=True), df2.loc[[match_index], :].reset_index(drop=True)], axis=1)
                result['Score'] = match[1]
                results.append(result)
                # Add the 'Acc' value to the set of matched ids
                matched_ids.add(row['Acc'])
            else:
                # If no match is found or the Score is below 70%, add the row from df1 to the results list
                result = df1.loc[[index], :].reset_index(drop=True)
                result['Score'] = None
                results.append(result)
# Concatenate all the results into a single dataframe
final_result = pd.concat(results)

In [ ]:
final_result.head(45)
# final_result.count()

In [ ]:
# Split the results into separate dataframes based on the Score score
above_90 = final_result[final_result['Score'] >= 90]
above_80 = final_result[(final_result['Score'] >= 80) & (final_result['Score'] < 90)]
above_70 = final_result[(final_result['Score'] >= 70) & (final_result['Score'] < 80)]
below_70 = final_result[final_result['Score'] < 70]
below_69 = below_70[below_70['Score'] < 69]

In [ ]:
# Check if the number of unique 'Acc' values in final_result matches the number of rows in df1
if final_result['Acc'].nunique() == df1.shape[0]:
    print("Result contains a row for every 'Acc' in df1")
else:
    print("Result does not contain a row for every 'Acc' in df1")

In [ ]:
above_90.count(),above_90_1.count(),above_90_2.count()


In [ ]:
# Write each dataframe to a separate CSV file
#Save each dataframe to a separate CSV file
above_90.to_csv('./Data_Result/sample_data_above_90.csv', index=False, encoding='utf-8')
above_80.to_csv('./Data_Result/sample_data_above_80.csv', index=False, encoding='utf-8')
above_70.to_csv('./Data_Result/sample_data_above_70.csv', index=False, encoding='utf-8')
below_69.to_csv('./Data_Result/sample_data_below_69.csv', index=False, encoding='utf-8')
final_result.to_csv('./Data_Result/sample_data_Fuzzy_result.csv', encoding='utf-8')
final_result.to_csv('./Data/sample_data_Fuzzy_result.csv', encoding='utf-8')